In [12]:
pip install rank-bm25

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for rank-bm25 from https://files.pythonhosted.org/packages/2a/21/f691fb2613100a62b3fa91e9988c991e9ca5b89ea31c0d3152a3210344f9/rank_bm25-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/7c/2f/244643a5ce54a94f0a9a2ab578189c061e4a87c002e037b0829dd77293b6/numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 649.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 11.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.4/12.8 MB 15.1 MB/s eta 0:00:01
   ------- -------------------------------- 2.3/12.8 MB 16.5 

In [ ]:
import pandas as pd
import sys
import numpy as np


Load Book as txt File

In [ ]:
import re

# Load the book's text
with open('data/TKMBFullBook.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Split the book into chapters first
# This regex splits the text on "Chapter" followed by a number
chapters = re.split(r'Chapter \d+', text)
if chapters[0].strip() == "":
    chapters.pop(0) # Remove the empty string before the first chapter


def normalize_punctuation(text: str) -> str:
 
    # This function normalizes punctuation and whitespace in the text
    #normalizing curly apostrophes
    text = text.replace("’", "'").replace("‘", "'")
    # Removing multi-dots to a single space (...)
    text = re.sub(r'\.{2,}', ' ', text)
    # Replacing hyphens/slashes with spaces (so “cross-road” → “cross road”)
    text = re.sub(r'[-/]', ' ', text)
    # Removing all punctuation except apostrophes
    # Keeping letters, numbers, whitespace, and apostrophes for contractions/possessives
    text = re.sub(r"[^\w\s']+", '', text)

    # Normalizing whitespace and lowercasing
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

structured_chunks = []
current_chapter = 1 # Start chapter numbering from 1

for chapter_text in chapters:
    # Split each chapter into paragraphs
    paragraphs = [p.strip() for p in chapter_text.split('\n\n') if p.strip()]
    
    for i, paragraph_text in enumerate(paragraphs):
        structured_chunks.append({
            'chapter': current_chapter, # Remove leading/trailing whitespace and convert to lowercase
            'paragraph_num': i + 1,
            'text': paragraph_text #.strip().lower() # Remove leading/trailing whitespace
        })
    current_chapter += 1

print(f"Created {len(structured_chunks)} structured chunks.")
print("Example chunk:")
print(structured_chunks[0])

Created 31 structured chunks.
Example chunk:
{'chapter': 1, 'paragraph_num': 1, 'text': 'When he was nearly thirteen, my brother Jem got his arm badly broken at the \nelbow. When it healed, and Jem’s fears of never being able to play football were \nassuaged, he was seldom self-conscious about his injury. His left arm was \nsomewhat shorter than his right; when he stood or walked, the back of his hand \nwas at right angles to his body, his thumb parallel to his thigh. He couldn’t have \ncared less, so long as he could pass and punt.\n When enough years had gone by to enable us to look back on them, we \nsometimes discussed the events leading to his accident. I maintain that the Ewells \nstarted it all, but Jem, who was four years my senior, said it started long before \nthat. He said it began the summer Dill came to us, when Dill first gave us the idea \nof making Boo Radley come out.\n I said if he wanted to take a broad view of the thing, it really began with Andrew \nJackson. If Gen

Lemmatizaion of words for better context

In [128]:

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> list[str]:
    
      #normalizing curly apostrophes
    text = text.replace("’", "'").replace("‘", "'")
    # Removing multi-dots to a single space (...)
    text = re.sub(r'\.{2,}', ' ', text)
    # Replacing hyphens/slashes with spaces (so “cross-road” → “cross road”)
    text = re.sub(r'[-/]', ' ', text)
    # Removing all punctuation except apostrophes
    # Keeping letters, numbers, whitespace, and apostrophes for contractions/possessives
    text = re.sub(r"[^\w\s']+", '', text)
    # Normalizing whitespace and lowercasing
    text = re.sub(r'\s+', ' ', text).strip().lower()
    
    # Tokenize the text into words
    tokens = word_tokenize(text.lower())  # Convert to lowercase for consistency
    # Lemmatize each word
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Return the list of lemmatized tokens
    return lemmatized_tokens

In [26]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\salij\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [129]:
import nltk
from rank_bm25 import BM25Okapi

# Create a list of lemmatized texts for BM25
Corpus = [lemmatize_text(chunk['text']) for chunk in structured_chunks]

# Initialize BM25 with the corpus for ranking and retrieval
bm25 = BM25Okapi(Corpus)

In [130]:
Corpus[1]  # Example of a lemmatized text

['dill',
 'left',
 'u',
 'early',
 'in',
 'september',
 'to',
 'return',
 'to',
 'meridian',
 'we',
 'saw',
 'him',
 'off',
 'on',
 'the',
 'five',
 "o'clock",
 'bus',
 'and',
 'i',
 'wa',
 'miserable',
 'without',
 'him',
 'until',
 'it',
 'occurred',
 'to',
 'me',
 'that',
 'i',
 'would',
 'be',
 'starting',
 'to',
 'school',
 'in',
 'a',
 'week',
 'i',
 'never',
 'looked',
 'forward',
 'more',
 'to',
 'anything',
 'in',
 'my',
 'life',
 'hour',
 'of',
 'wintertime',
 'had',
 'found',
 'me',
 'in',
 'the',
 'treehouse',
 'looking',
 'over',
 'at',
 'the',
 'schoolyard',
 'spying',
 'on',
 'multitude',
 'of',
 'child',
 'through',
 'a',
 'two',
 'power',
 'telescope',
 'jem',
 'had',
 'given',
 'me',
 'learning',
 'their',
 'game',
 'following',
 'jem',
 "'s",
 'red',
 'jacket',
 'through',
 'wriggling',
 'circle',
 'of',
 'blind',
 'man',
 "'s",
 'buff',
 'secretly',
 'sharing',
 'their',
 'misfortune',
 'and',
 'minor',
 'victory',
 'i',
 'longed',
 'to',
 'join',
 'them',
 'jem',
 

LLM to Query

In [155]:
import openai
from openai import OpenAI
import os

# Set up OpenAI API key
#openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this environment variable

client = OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

#generate text
def generate_text(prompt: str) -> str:
    """Generates text using the OpenAI API based on the provided prompt."""
    
    # Make the API call
    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # A powerful and cost-effective choice
        messages=[{"role": "system", "content": "You are a creative short story writer who writes stories in a specified book style and answer questions based on it."},
                  {"role": "user", "content": prompt}],
        temperature=0.7,  # Adjust temperature for creativity
    )
    
    return response.choices[0].message.content.strip()

In [190]:

#generated_text = generate_text("In 100 words, write a short story about a summer day in Manchester, inspired by the style of 'To Kill a Mockingbird'.") # Dill would be leaving for Meridian
generated_text = generate_text("In a few 100 words write a short story of how innocence can be confused with guilt in a style of Harper Lee.")
generated_text

'The summer sun hung low over Maycomb, casting long shadows through the dusty streets and swaying branches of ancient oaks. Jem and Scout Finch sat on the porch steps, their bare feet dangling, watching the world with wide, curious eyes. It was the kind of afternoon where time seemed to slow, yet the air was thick with a tension they couldn’t quite name.\n\nTommy Harris was new in town, a shy boy with a crooked smile who kept to himself. One afternoon, a window shattered at the church, and fingers quickly pointed toward Tommy. Rumors swirled — some said he was clumsy, others whispered darker things. But Jem and Scout, knowing the boy’s quiet kindness, couldn’t believe he was guilty of such mischief.\n\nAtticus, their father, sat stiffly at the kitchen table, the weight of the town’s silent judgments pressing down on him. “Sometimes,” he said, “people see what they expect to see, not what is true.” His voice was calm, but Scout caught the flicker of sadness in his eyes.\n\nThe trial was

In Each chapter get an array of the BM25 scores indicating similarity

In [191]:

query = generated_text
processed_query = lemmatize_text(query)

doc_scores = bm25.get_scores(processed_query)

In [192]:
doc_scores

array([262.51151431, 232.74970405, 245.49131769, 246.5230059 ,
       242.10936822, 249.31219282, 233.74739871, 243.09146797,
       245.15438729, 240.97227065, 249.29130303, 244.21125725,
       244.91926861, 240.76466579, 254.76689571, 247.811424  ,
       245.40532226, 243.38821679, 245.92997477, 241.77868495,
       240.31158316, 224.71897551, 243.82195551, 251.28608205,
       222.90101043, 230.63397191, 227.40408917, 259.0522676 ,
       222.67528351, 246.04628143, 227.97060655])

Querying BM25 Retrival to get passages from the book

In [194]:
# Get the top 2 documents that match the query and print the structured chunks
# Get BM25 scores for all documents first
doc_scores = bm25.get_scores(processed_query)

# Get the top 2 indices (not the token lists)
top_indices = doc_scores.argsort()[-2:][::-1]  # Get indices of top 2 scores in descending order

print(f"Found top 2 matching chunks:\n")

for rank, idx in enumerate(top_indices):
    chunk = structured_chunks[idx]
    bm25_score = doc_scores[idx]
    
    print(f"Rank {rank + 1}:")
    print(f"'chapter': {chunk['chapter']}, 'paragraph_num': {chunk['paragraph_num']}, 'bm25_score': {bm25_score:.4f}")
    #print(f"'text': {chunk['text']}")
    print("-" * 80)

Found top 2 matching chunks:

Rank 1:
'chapter': 1, 'paragraph_num': 1, 'bm25_score': 262.5115
--------------------------------------------------------------------------------
Rank 2:
'chapter': 28, 'paragraph_num': 1, 'bm25_score': 259.0523
--------------------------------------------------------------------------------


Evaluating the BM25 retrived results

In [169]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model (this is done once)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Inside your evaluation loop...
llm_answer = "Dill left us early in September, to return to Meridian. We saw him off on the five o’clock bus and I was miserable without him until the day I found a small, folded note tucked beneath my bedroom window. It was in Dill’s familiar scrawl, promising he’d be back before the next summer, with stories of faraway places and new adventures. That note became my secret treasure, a beacon through the dull months. I clung to it, imagining Dill’s voice, his laughter, and the promise of our friendship rekindled, reminding me that some bonds stretch beyond distance and time."
ground_truth_text = "Dill left us early in September, to return to Meridian. We saw him off on the five o’clock bus and I was miserable without him until it occurred to me that I would be starting to school in a week. I never looked forward more to anything in my life. Hours of wintertime had found me in the treehouse, looking over at the schoolyard, spying on multitudes of children through a two-power telescope Jem had given me, learning their games, following Jem’s red jacket through wriggling circles of blind man’s buff, secretly sharing their misfortunes and minor victories. "

# Convert texts to embeddings
embedding1 = model.encode(llm_answer, convert_to_tensor=True)
embedding2 = model.encode(ground_truth_text, convert_to_tensor=True)

# Compute cosine similarity
cosine_score = util.cos_sim(embedding1, embedding2)

print(f"Semantic Similarity Score: {cosine_score.item():.4f}")


Semantic Similarity Score: 0.7266


c:\Users\salij\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
